## 네이버 블로그 크롤링하기

먼저 github에 있는 https://github.com/xotrs/naver-blog-crawler 의 코드를 사용하여 크롤링을 시도해본다. 

In [20]:
naver_client_id = "XZWzIZhcSDod0cvL9MbH"
naver_client_secret = "a7yabVH_9c"

In [23]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup


def naver_blog_crawling(search_blog_keyword, display_count, sort_type):
    search_result_blog_page_count = get_blog_search_result_pagination_count(search_blog_keyword, display_count)
    get_blog_post(search_blog_keyword, display_count, search_result_blog_page_count, sort_type)


def get_blog_search_result_pagination_count(search_blog_keyword, display_count):
    encode_search_keyword = urllib.parse.quote(search_blog_keyword)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_keyword
    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code is 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))

        if response_body_dict['total'] == 0:
            blog_pagination_count = 0
        else:
            blog_pagination_total_count = math.ceil(response_body_dict['total'] / int(display_count))

            if blog_pagination_total_count >= 1000:
                blog_pagination_count = 1000
            else:
                blog_pagination_count = blog_pagination_total_count

            print("키워드 " + search_blog_keyword + "에 해당하는 포스팅 수 : " + str(response_body_dict['total']))
            print("키워드 " + search_blog_keyword + "에 해당하는 블로그 실제 페이징 수 : " + str(blog_pagination_total_count))
            print("키워드 " + search_blog_keyword + "에 해당하는 블로그 처리할 수 있는 페이징 수 : " + str(blog_pagination_count))

        return blog_pagination_count


def get_blog_post(search_blog_keyword, display_count, search_result_blog_page_count, sort_type):
    encode_search_blog_keyword = urllib.parse.quote(search_blog_keyword)

    for i in range(1, search_result_blog_page_count + 1):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_blog_keyword + "&display=" + str(
            display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)

        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        response = urllib.request.urlopen(request)
        response_code = response.getcode()

        if response_code is 200:
            response_body = response.read()
            response_body_dict = json.loads(response_body.decode('utf-8'))

            for j in range(0, len(response_body_dict['items'])):
                try:
                    blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")

                    get_blog_post_content_code = requests.get(blog_post_url)
                    get_blog_post_content_text = get_blog_post_content_code.text

                    get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')

                    for link in get_blog_post_content_soup.select('frame#mainFrame'):
                        real_blog_post_url = "http://blog.naver.com" + link.get('src')

                        get_real_blog_post_content_code = requests.get(real_blog_post_url)
                        get_real_blog_post_content_text = get_real_blog_post_content_code.text

                        get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

                        for blog_post_content in get_real_blog_post_content_soup.select('div#postViewArea'):
                            blog_post_content_text = blog_post_content.get_text()

                            remove_html_tag = re.compile('<.*?>')

                            blog_post_title = re.sub(remove_html_tag, '', response_body_dict['items'][j]['title'])
                            blog_post_description = re.sub(remove_html_tag, '',
                                                           response_body_dict['items'][j]['description'])
                            blog_post_postdate = datetime.datetime.strptime(response_body_dict['items'][j]['postdate'],
                                                                            "%Y%m%d").strftime("%y.%m.%d")
                            blog_post_blogger_name = response_body_dict['items'][j]['bloggername']
                            blog_post_full_contents = str(blog_post_content_text)

                            print("포스팅 URL : " + blog_post_url)
                            print("포스팅 제목 : " + blog_post_title)
                            print("포스팅 설명 : " + blog_post_description)
                            print("포스팅 날짜 : " + blog_post_postdate)
                            print("블로거 이름 : " + blog_post_blogger_name)
                            print("포스팅 내용 : " + blog_post_full_contents)
                except:
                    j += 1

In [24]:
naver_blog_crawling("나노하나", 10, "date")

HTTPError: HTTP Error 401: Unauthorized

네이버에서 어플리케이션을 등록하고 id와 secret_key를 넣었는데도 401 에러가난다. 

그럼 다음
https://gist.github.com/allieus/614c732941b3e3725b595f787a1efc57
의 코드를 적용해본다. 

askDjango 커뮤니티를 운영하고, 파이썬 강의도 많이 하는 이진석님의 코드니까 좀 더 신뢰가 간다. 일단 주석을 달면서 코드를 파악해보기

In [34]:
import re
import itertools
import time
import requests
from datetime import date
from collections import namedtuple, OrderedDict
from bs4 import BeautifulSoup


Category = namedtuple('Category', ['unique_id', 'name'])
Post = namedtuple('Post', ['unique_id', 'url', 'title', 'categories', 'tags', 'html', 'posted_at'])


class NaverBlog(object):
    # blog_id와 page를 넣어서 크롤링해오나보다. 
    POST_LIST_URL_FORMAT = 'http://blog.naver.com/PostList.nhn?blogId={blog_id}&currentPage={page}'

    # NaverBlog 객체를 부를 때 초기화되는 듯?
    def __init__(self, blog_id):
        self.blog_id = blog_id
        # blog id가 'blog.naver.com/PostList.nhn' 얘를 포함하는 경우
        if 'blog.naver.com/PostList.nhn' in self.blog_id:
            #blog_id= 뒤에 오는 애를 blog_id로 간주
            matched = re.search(r'\?blogId=([a-zA-Z0-9_]+)', self.blog_id)
            if matched:
                self.blog_id = matched.group(1)
        # blog id가 'blog.naver.com' 얘를 포함하는 경우
        elif 'blog.naver.com' in self.blog_id:
            #blog.naver.com/ 뒤에 오는 애를 blog_id로 간주
            matched = re.search(r'blog.naver.com/([a-zA-Z0-9_]+)', self.blog_id)
            if matched:
                self.blog_id = matched.group(1)

    def get_post_list(self):
        #  OrderedDict : insertion order를 기억하는 dict
        post_dict = OrderedDict()

        for page in itertools.count(start=1):
            #포맷에 blog_id와 page를 넣어주고
            url = self.POST_LIST_URL_FORMAT.format(blog_id=self.blog_id, page=page)
            print('getting {}'.format(url))
            #python은 try except else를 쓴다. java의 try catch를 생각하면 되는듯
            try:
                r = requests.get(url)
                #HTTP request가 성공이 아닌 response를 받아오면 HTTPError를 일으킨다
                r.raise_for_status()
            except requests.HTTPError as e:
                print(e)
                break
            else:
                # 성공적인 response를 받아왔을 경우에는
                soup = BeautifulSoup(r.text, 'html.parser')
                #Compile a regular expression pattern into a regular expression object, 
                # which can be used for matching using its match(), search() and other methods
                # re.compile하면 정규표현식을 객체로 만들어주고 match(), search() 등에서 쓸 수 있게 해준대!
                post_tag_idx_pattern = re.compile(r'post_(\d+)')

                post_tags = soup.findAll(id=post_tag_idx_pattern)

                if not post_tags:
                    break

                # 블로그의 내용을 전부 가져오고
                # 그 중 post_숫자들 의 형태로 포스팅이 있을 경우에는
                for post_tag in post_tags:
                    post_idx = re.match(post_tag_idx_pattern, post_tag.attrs['id']).group(1)

                    title_tag = post_tag.find(id='title_' + post_idx)
#                     print('title_tag : ', title_tag)
                    try:
                        category_tag = title_tag.select('.cate')[0]
                        category_tag.extract()
                        title = ' '.join(title_tag.text.split())
#                         print('title_tag.text.split() : ', title_tag.text.split())
                        print('title : ', title)
                        # 찍어봐도 왜 이렇게 title이 되는지 잘 모르겠다... 일단 타이틀을 이렇게 뽑아온다, 패스!

                        categories = []
                        for tag in category_tag.findAll('a'):
                            unique_id = re.search('categoryNo=(\d+)', tag.attrs['href']).group(1)
                            name = ' '.join(tag.text.split())
                            categories.append(Category(unique_id, name))

                        year, month, day = map(int, re.findall(r'\d+', post_tag.select('.date')[0].text)[:3])
                        posted_at = date(year, month, day)

                        url = post_tag.select('.url a')[0].attrs['href']
                        unique_id = re.search(r'(\d+)$', url).group(1)

                        html = str(post_tag.select('.post-view')[0])

                        # 이런식으로 태그를 가지고 posted_at, category를 가져오는구나

                        # TODO: naver blog 에서 ajax loading 을 시키기 때문에 로딩이 안 됨 ㅠ_ㅠ
                        # 위의 주석은 무슨 말이냐! 모름 패스...
                        tags = []
                        for tag_tag in post_tag.select('.post-tag a.tag'):
                            name = ' '.join(tag_tag.text.split())
                            tags.append(name)

                        post = Post(unique_id=unique_id, url=url, title=title, categories=categories, tags=tags, html=html, posted_at=posted_at)
                        if post.unique_id in post_dict:
                            print('duplicated post. quit.')
                            return None

                        post_dict[post.unique_id] = post
                        yield post

                    except AttributeError:
                        print('attr err')
                    
                time.sleep(0.5)


if __name__ == '__main__':
    # 'doginsight'라는 블로그에서
    blog = NaverBlog('doginsight')
    # 모든 포스트를 가져온다
    for idx, post in enumerate(blog.get_post_list()):
        print(idx+1, post.title, post.posted_at, post.url)

getting http://blog.naver.com/PostList.nhn?blogId=doginsight&currentPage=1
title :  [도그인사이트] 동물 복지, 실현 가능한 걸까
1 [도그인사이트] 동물 복지, 실현 가능한 걸까 2017-07-29 http://blog.naver.com/doginsight/221062530939
attr err
title :  [도그인사이트] 말티즈
2 [도그인사이트] 말티즈 2017-05-09 http://blog.naver.com/doginsight/221001508111
getting http://blog.naver.com/PostList.nhn?blogId=doginsight&currentPage=2
title :  [도그인사이트] 치와와
3 [도그인사이트] 치와와 2017-01-29 http://blog.naver.com/doginsight/220922295506
title :  [도그인사이트] 닥스훈트
4 [도그인사이트] 닥스훈트 2017-01-21 http://blog.naver.com/doginsight/220916929356
title :  [도그인사이트] 포메라니안
5 [도그인사이트] 포메라니안 2017-01-17 http://blog.naver.com/doginsight/220913740564
getting http://blog.naver.com/PostList.nhn?blogId=doginsight&currentPage=3
title :  [도그인사이트] 푸들
6 [도그인사이트] 푸들 2017-01-08 http://blog.naver.com/doginsight/220905576735
title :  [도그인사이트] 신림 지니앤독 스티커 이벤트
7 [도그인사이트] 신림 지니앤독 스티커 이벤트 2016-08-07 http://blog.naver.com/doginsight/220781466356
title :  [도그인사이트] 수박 상세
8 [도그인사이트] 수박 상세 2016-06-10 ht

KeyboardInterrupt: 

우리는 특정 블로그를 지정해서 뭔가를 가져오는 것이 아니라 키워드와 기간을 통해서 검색을 하고 싶은데... 원하는 바와는 다른 코드이다.

그래도 네이버 블로그가 blog.naver.com/blog_id의 형식과 \?blogId=blog_id의 두 가지 형식을 가진다는 것을 알 수 있었다. 
그냥 내가 해보자

In [1]:
from selenium import webdriver

In [2]:
driver = webdriver.Chrome('C:/chromedriver.exe')

In [3]:
driver.implicitly_wait(3)
# url에 접근한다.
driver.get('https://search.naver.com/search.naver?where=post&query=%EB%82%98%EB%85%B8%ED%95%98%EB%82%98&st=sim&sm=tab_opt&date_from=20170305&date_to=20170505&date_option=8&srchby=all&dup_remove=1&post_blogurl=&post_blogurl_without=&nso=so%3Ar%2Ca%3Aall%2Cp%3Afrom20170305to20170505&mson=0')